# IVVI Rack Signal Chain Example

This notebook demonstrates the usage of the modular signal chain virtual instrument for IVVI rack topology.

## Signal Chain Topology

```
[MFLI AC Voltage Source (programmable V)]
  → [Manual V→I Transformer (manual transconductance)]
  → [Sample]
  → [Manual Voltage Preamp (flat V/V)]
  → [MFLI Input & Demod (programmable)]
```

The key feature is a single current setpoint parameter (`I_set`) that automatically computes and applies the correct MFLI source voltage amplitude.

In [ ]:
# Import required modules
import numpy as np
from qcodes.parameters import ManualParameter

# Import signal chain components
from qcodes_contrib_drivers.drivers.ZurichInstruments.MFLI import MFLI
from qcodes_contrib_drivers.drivers.virtual import (
    MFLISource, 
    ManualVITransformer, 
    ManualVoltagePreamp, 
    MFLILockIn,
    SignalChain
)

## Setup Signal Chain Components

In a real setup, you would connect to actual MFLI hardware. For this example, we'll use mock parameters.

In [ ]:
# For demonstration, create mock MFLI parameters
# In real use, you would connect to actual hardware:
# mfli_hw = MFLI('mfli', 'dev1234', demod=0, sigout=0, auxouts={'X': 0, 'Y': 1, 'R': 2, 'Theta': 3})

class MockMFLI:
    """Mock MFLI for demonstration purposes."""
    def __init__(self):
        self.amplitude = ManualParameter('amplitude', initial_value=0.001, unit='V')
        self.frequency = ManualParameter('frequency', initial_value=1000.0, unit='Hz')
        self.output_on = ManualParameter('output_on', initial_value=False)
        self.time_constant = ManualParameter('time_constant', initial_value=0.01, unit='s')
        self.sensitivity = ManualParameter('sensitivity', initial_value=1e-3, unit='V')
        self.input_range = ManualParameter('input_range', initial_value=1.0, unit='V')
        self.X = ManualParameter('X', initial_value=0.0, unit='V')
        self.Y = ManualParameter('Y', initial_value=0.0, unit='V')
        self.R = ManualParameter('R', initial_value=0.0, unit='V')
        self.Theta = ManualParameter('Theta', initial_value=0.0, unit='deg')

mock_mfli = MockMFLI()

# Create signal chain nodes
src_v = MFLISource('mfli_source', mock_mfli)
vi_transformer = ManualVITransformer('vi_transformer')
preamp = ManualVoltagePreamp('voltage_preamp')
lockin = MFLILockIn('mfli_lockin', mock_mfli)

print("Signal chain components created successfully")

## Configure Manual Parameters

Set the manual parameters that describe your physical setup.

In [ ]:
# Configure V→I transformer
vi_transformer.gm_a_per_v.set(1e-3)  # 1 mA/V transconductance
vi_transformer.invert.set(False)      # No polarity inversion

# Configure voltage preamp
preamp.gain_v_per_v.set(100.0)      # 100 V/V gain
preamp.invert.set(False)             # No polarity inversion

print(f"V→I transconductance: {vi_transformer.gm_a_per_v()} A/V")
print(f"Preamp gain: {preamp.gain_v_per_v()} V/V")

## Create Signal Chain Virtual Instrument

The signal chain provides a unified interface with current setpoint control.

In [ ]:
# Create the signal chain virtual instrument
ivvi_rack = SignalChain(
    src_v=src_v,
    vi_transformer=vi_transformer, 
    preamp=preamp,
    lockin=lockin,
    name='ivvi_rack'
)

# Configure advisory parameters
ivvi_rack.R_est.set(10e3)        # Estimate 10 kΩ sample resistance
ivvi_rack.margin.set(3.0)        # 3× safety margin for guards
ivvi_rack.amplitude_convention.set('rms')  # Use RMS convention

print("IVVI rack signal chain created successfully")
print(f"Available parameters: {list(ivvi_rack.parameters.keys())}")

## Demonstrate Current Setpoint Control

The key feature: set a target current and the system automatically calculates the required source voltage.

In [ ]:
# Set target current
I_target = 1e-6  # 1 µA

print(f"Setting target current: {I_target*1e6:.1f} µA")
ivvi_rack.I_set.set(I_target)

# Check what happened
print(f"Excitation voltage set to: {ivvi_rack.excitation_v_ac()*1e3:.3f} mV RMS")
print(f"Output enabled: {ivvi_rack.output_on()}")
print(f"Commanded current: {ivvi_rack.I_cmd()*1e6:.1f} µA")

# Verify calculation
gm_eff = ivvi_rack._gm_eff()
expected_voltage = I_target / gm_eff
print(f"Expected voltage (I/gm): {expected_voltage*1e3:.3f} mV")

## Frequency Coupling

Setting the reference frequency updates both source and demod frequencies simultaneously.

In [ ]:
# Set reference frequency
freq = 1500.0  # Hz
ivvi_rack.reference_frequency.set(freq)

print(f"Reference frequency set to: {freq} Hz")
print(f"Source frequency: {src_v.frequency()} Hz")
print(f"Lock-in frequency: {lockin.frequency()} Hz")
print(f"Reference frequency readback: {ivvi_rack.reference_frequency()} Hz")

## Simulate a Measurement

Simulate taking a measurement and see the derived parameters in action.

In [ ]:
# Simulate measurement data (in a real setup, this comes from hardware)
# Let's simulate that we measure the expected response
R_sample = 10e3  # 10 kΩ as estimated
I_actual = I_target  # Assume we get exactly what we set
V_sample_actual = I_actual * R_sample  # Ohm's law
V_preamp_out = V_sample_actual * preamp.gain_v_per_v()  # After preamp

# Set the lock-in readings to simulate this
mock_mfli.X.set(V_preamp_out)  # Only X component for simplicity
mock_mfli.Y.set(0.0)
mock_mfli.R.set(V_preamp_out)
mock_mfli.Theta.set(0.0)

print(f"Simulated measurement:")
print(f"  Sample voltage: {V_sample_actual*1e3:.3f} mV")
print(f"  Preamp output: {V_preamp_out:.3f} V")
print(f"  Lock-in X: {ivvi_rack.X():.3f} V")
print(f"  Lock-in Y: {ivvi_rack.Y():.3f} V")
print(f"  Lock-in R: {ivvi_rack.R():.3f} V")

## Derived Physics Parameters

The signal chain automatically calculates physics-relevant parameters.

In [ ]:
# Check derived parameters
V_sample_meas = ivvi_rack.V_sample_ac_meas()
I_meas = ivvi_rack.I_meas()
recommended_sens = ivvi_rack.recommended_sensitivity()

print(f"Derived parameters:")
print(f"  Measured sample voltage: {abs(V_sample_meas)*1e3:.3f} mV")
print(f"  Measured current: {I_meas*1e6:.1f} µA")
print(f"  Recommended sensitivity: {recommended_sens:.3f} V")
print(f"  Current sensitivity: {ivvi_rack.sensitivity():.3g} V")

# Verify the calculations
print(f"\nVerification:")
print(f"  V_sample expected: {V_sample_actual*1e3:.3f} mV")
print(f"  V_sample measured: {abs(V_sample_meas)*1e3:.3f} mV")
print(f"  I_target: {I_target*1e6:.1f} µA")
print(f"  I_measured: {I_meas*1e6:.1f} µA")

## Guard Functions

Test the guard functions that warn about potential overloads.

In [ ]:
# Test guard function by setting a large current that would overload
print("Testing guard function...")
print(f"Current input range: {ivvi_rack.input_range()} V")

# Calculate current that would just exceed 80% of input range
input_range = ivvi_rack.input_range()
R_est = ivvi_rack.R_est()
gv_eff = abs(ivvi_rack._gv_eff())

I_80_percent = 0.8 * input_range / (R_est * gv_eff)
I_overload = 0.9 * input_range / (R_est * gv_eff)  # This should trigger warning

print(f"Current for 80% of range: {I_80_percent*1e6:.2f} µA")
print(f"Testing with: {I_overload*1e6:.2f} µA (should trigger guard)")

# This should print a guard warning
ivvi_rack.I_set.set(I_overload)

## Snapshot and State Management

The signal chain properly captures all manual parameters in snapshots.

In [ ]:
# Take a snapshot of the entire signal chain
snapshot = ivvi_rack.snapshot()

print("Signal chain snapshot (key parameters):")
for param in ['I_set', 'I_cmd', 'excitation_v_ac', 'reference_frequency', 
              'gm_a_per_v', 'preamp_gain', 'R_est', 'margin']:
    if param in snapshot['parameters']:
        value = snapshot['parameters'][param]['value']
        unit = snapshot['parameters'][param].get('unit', '')
        print(f"  {param}: {value} {unit}")

## Summary

This example demonstrated:

1. **Modular Architecture**: Abstract bases + concrete nodes for different devices
2. **Current Setpoint Control**: Single `I_set` parameter with automatic voltage calculation
3. **Frequency Coupling**: Synchronized source and demod frequencies
4. **Derived Parameters**: Automatic calculation of physics-relevant quantities
5. **Guard Functions**: Protection against input overloads
6. **State Management**: Proper snapshotting of manual parameters

### Key Features:

- **I_set = 1e-6 A** → automatically sets **excitation_v_ac = 1e-3 V** (for gm = 1e-3 A/V)
- **reference_frequency** couples source and demod frequencies
- **Guard functions** warn when predicted input exceeds 80% of range
- **Derived parameters** provide V_sample_ac_meas, I_meas, recommended_sensitivity
- **Manual parameters** are properly captured in snapshots

The system is designed to be **minimal disruption** to existing code while providing a **clean, physics-oriented interface** for current-based measurements.